In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
df

# **Removing HTML**

In [ ]:
df.iloc[0]['review']

In [ ]:
import re

CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [ ]:
from bs4 import BeautifulSoup
from tqdm import trange

for i in trange(0,len(df)) :
    if bool(BeautifulSoup(df.iloc[i]['review'], "html.parser").find()) :
        df.at[i,'review'] = cleanhtml(df.iloc[i]['review'])

In [ ]:
df

# Remove Urls

In [ ]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [ ]:
df['review'] = df['review'].apply(remove_url)

In [ ]:
df

# Remove Chat words used

In [ ]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [ ]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

In [ ]:
chat_words_map_dict

In [ ]:
count = 0
last_saved = ''
for i in trange(0,len(df['review'])) :  
    if any(word in df.iloc[i]['review'] for word in chat_words_list):
        count = count + 1
        last_saved = df.iloc[i]['review']
        
count

In [ ]:
print(last_saved)

for word in chat_words_list :
    if word in last_saved :
        print("\n",word)


In [ ]:
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
df['review'] = df['review'].apply(chat_words_conversion)

df

# Spelling Corrections

In [ ]:
# from textblob import TextBlob

# def correct(text):
#     textblb = TextBlob(text) 
#     return textblb.correct().string

In [ ]:
# for i  in trange(0,len(df['review'])) :
#     df.at[i,'review'] = correct(df.iloc[i]['review'])

# df

# Removing digits and emojis

In [ ]:
import re

def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
df['review'] = df['review'].apply(remove_special_characters)

df

In [ ]:
df.at[50000,'review'] = 'The movie was not good enough.'
df.at[50001,'review'] = 'The movie was not good.'

df.at[50000,'sentiment'] = 'negative'
df.at[50001,'sentiment'] = 'negative'

df

# **Text Processing**

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk

In [ ]:
df["text_lower"] = df["review"].str.lower()
df

In [ ]:
tt = TweetTokenizer()
df['tokenized'] = df['text_lower'].apply(tt.tokenize)
df

In [ ]:
stop_words = stopwords.words('english')

print(stopwords)

In [ ]:
df['stopwords'] = df['tokenized'].apply(lambda x: list([word for word in x if word not in (stop_words)]))
df

In [ ]:
stem = PorterStemmer()
df['stemming'] = df['tokenized'].apply(lambda x: list([stem.stem(word) for word in x]))
df

In [ ]:
lem = WordNetLemmatizer()
df['lemmatize'] = df['tokenized'].apply(lambda x: list([lem.lemmatize(word) for word in x]))
df.head()

# **Word cloud**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

In [ ]:
all_stemming = df['stemming'].apply(lambda x: " ".join(word for word in x))
all_text_stemming = ' '.join(all_stemming)
print(type(all_text_stemming))

In [ ]:
all_lemetized = df['lemmatize'].apply(lambda x: " ".join(word for word in x))
all_text_lemetized = ' '.join(all_lemetized)
print(type(all_text_lemetized))

In [ ]:
word_cloud1 = WordCloud(collocations = False,background_color = 'white').generate(all_text_stemming)
word_cloud2 = WordCloud(collocations = False,background_color = 'white').generate(all_text_lemetized)

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(100,100))
fig.add_subplot(1, 2, 1)
plt.imshow(word_cloud1)
plt.axis('off')
plt.title("stemming")

fig.add_subplot(1, 2, 2)
plt.imshow(word_cloud2)
plt.axis('off')
plt.title("lemetized")

plt.show()

# **Final data for Model**

In [ ]:
for i in trange(0,len(df)) :
    df.at[i,'final_ml'] = ' '.join([str(elem) for elem in df.iloc[i]['lemmatize']])
    df.at[i,'final_dl'] = ' '.join([str(elem) for elem in df.iloc[i]['lemmatize']])
    
df

In [ ]:
X = df['final_ml']
Y = df['sentiment']

In [ ]:
X

# **Label Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y = le.fit_transform(Y)

Y

# **Train test split**

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train, y_test = train_test_split(X,Y,test_size = 0.2,shuffle=True)

# **Making count vectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(x_train)

In [ ]:
x_train = cv.transform(x_train)
x_test = cv.transform(x_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_for = RandomForestClassifier(n_estimators=101, criterion='entropy', max_depth=250)
model_for.fit(x_train,y_train)
y_pred = model_for.predict(x_test)
print(model_for.score(x_test,y_test))

# **Deep learning Models**

In [ ]:
from numpy import array
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X = df['final_dl']

In [ ]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(X)

In [ ]:
# tokenizer.word_index

In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1

VOCAB_SIZE

In [ ]:
df['sequences'] = tokenizer.texts_to_sequences(X)

df

In [ ]:
MAXLEN = max([len(sequence) for sequence in df['sequences']])

MAXLEN

In [ ]:
from sklearn.model_selection import train_test_split

x1_train, x1_test,y1_train, y1_test = train_test_split(df['sequences'],Y,test_size = 0.2,shuffle=True)

In [ ]:
x1_train

In [ ]:
x1_train = pad_sequences(x1_train, MAXLEN+1,padding='pre')
x1_test = pad_sequences(x1_test, MAXLEN+1,padding='pre')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(x1_train, y1_train, validation_data=(x1_test,y1_test), epochs=5)

In [ ]:
import matplotlib.pyplot as plt
import numpy

names_ = cv.get_feature_names()
importances = model_for.feature_importances_
# importances


sorted_indices = numpy.argsort(importances)[::-1]

print(sorted_indices)

print('-'*10 + ' Top 20 words which has more impact on model ' + '-'*10)

for i in range(0,20) :
  x = sorted_indices.tolist()[i]
  print(names_[x])

In [ ]:
import pickle

In [ ]:
with open(r'/kaggle/working/Random_forest_clf.pkl', 'wb') as file1:
    pickle.dump(model_for, file1)

In [ ]:
model.save(r'/kaggle/working/LSTM_rnn.h1')

In [ ]:
%cd /kaggle/working

from IPython.display import FileLinks
FileLinks(r'LSTM_rnn.h1')

In [ ]:
with open(r'/kaggle/working/cv.pkl', 'wb') as file2:
    pickle.dump(cv, file2)

In [ ]:
with open(r'/kaggle/working/tokenizer.pkl', 'wb') as file3:
    pickle.dump(tokenizer, file3)

In [ ]:
import keras

pretrained = keras.models.load_model('../input/pretrained-imdb/LSTM_rnn.h1')

pretrained.summary()